In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

## Extracting features of 2020 movies from Wikipedia

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [16]:
tables = soup.find_all('table',class_='wikitable sortable')

In [17]:
len(tables)

4

In [18]:
type(tables[0])

bs4.element.Tag

In [19]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [27]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [28]:
df.isnull().sum()

Opening               0
Opening.1             0
Title                 0
Production company    0
Cast and crew         0
Ref.                  8
dtype: int64

In [29]:
df = df.dropna(how="any")

In [30]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2]
1,JANUARY,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3]
2,JANUARY,10,Like a Boss,Paramount Pictures,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4]
3,JANUARY,10,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5]
4,JANUARY,10,The Sonata,Screen Media Films,Andrew Desmond (director/screenplay); Arthur M...,[6]
5,JANUARY,10,The Murder of Nicole Brown Simpson,Quiver Distribution,Daniel Farrands (director); Michael Arter (scr...,[7]
6,JANUARY,17,Bad Boys for Life,Columbia Pictures / Don Simpson/Jerry Bruckhei...,Adil El Arbi and Bilall Fallah (directors); Ch...,[8]
7,JANUARY,17,Dolittle,Universal Pictures / Perfect World Pictures,Stephen Gaghan (director/screenplay); Dan Greg...,[9]
8,JANUARY,17,A Fall from Grace,Netflix,"Tyler Perry (director); Crystal R. Fox, Phylic...",[10]
9,JANUARY,24,The Gentlemen,STX Films / Miramax,Guy Ritchie (director/screenplay); Matthew McC...,[11]


In [31]:
df_2020 = df[['Title','Cast and crew']]

In [32]:
df_2020

,Title,Cast and crew
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...
1,Underwater,"William Eubank (director); Brian Duffield, Ada..."
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col..."
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...
5,The Murder of Nicole Brown Simpson,Daniel Farrands (director); Michael Arter (scr...
6,Bad Boys for Life,Adil El Arbi and Bilall Fallah (directors); Ch...
7,Dolittle,Stephen Gaghan (director/screenplay); Dan Greg...
8,A Fall from Grace,"Tyler Perry (director); Crystal R. Fox, Phylic..."
9,The Gentlemen,Guy Ritchie (director/screenplay); Matthew McC...


In [33]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '04a88a6fef0997711c149ea513f8d6da'

In [52]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def get_genre(x):
    Genres = []
    result = tmdb_movie.search(x)
    try:
        movie_id = result[0].id
    except IndexError:
        return(np.NaN)
        
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
    data_json = response.json()
    if data_json['genres']:
        genre_str = " " 
        for i in range(0,len(data_json['genres'])):
            Genres.append(data_json['genres'][i]['name'])
        return genre_str.join(Genres)
    else:
        np.NaN

In [53]:
df_2020['Genres'] = df_2020['Title'].map(lambda x: get_genre(str(x)))

C:\Users\sam16\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [54]:
df_2020.isnull().sum()

Title            0
Cast and crew    0
Genres           1
dtype: int64

In [55]:
df = df.dropna(how="any")

In [56]:
df_2020

,Title,Cast and crew,Genres
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Drama Thriller Crime
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...,Horror Thriller Mystery
5,The Murder of Nicole Brown Simpson,Daniel Farrands (director); Michael Arter (scr...,Thriller Drama Crime
6,Bad Boys for Life,Adil El Arbi and Bilall Fallah (directors); Ch...,Thriller Action Crime
7,Dolittle,Stephen Gaghan (director/screenplay); Dan Greg...,Comedy Fantasy Adventure Family
8,A Fall from Grace,"Tyler Perry (director); Crystal R. Fox, Phylic...",Thriller
9,The Gentlemen,Guy Ritchie (director/screenplay); Matthew McC...,Action Crime Comedy


In [57]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [58]:
df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))

C:\Users\sam16\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [59]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [60]:
df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))

C:\Users\sam16\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [61]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [62]:
df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))

In [63]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [64]:
df_2020['actor_3_name'] = df_2020['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [65]:
df_2020

,Title,Cast and crew,Genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Drama Thriller Crime,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...,Horror Thriller Mystery,Andrew Desmond,Freya Tingley,Simon Abkarian,Rutger Hauer
5,The Murder of Nicole Brown Simpson,Daniel Farrands (director); Michael Arter (scr...,Thriller Drama Crime,Daniel Farrands,Mena Suvari,Nick Stahl,Taryn Manning
6,Bad Boys for Life,Adil El Arbi and Bilall Fallah (directors); Ch...,Thriller Action Crime,Adil El Arbi and Bilall Fallah,Will Smith,Martin Lawrence,Vanessa Hudgens
7,Dolittle,Stephen Gaghan (director/screenplay); Dan Greg...,Comedy Fantasy Adventure Family,Stephen Gaghan,Robert Downey Jr.,Antonio Banderas,Michael Sheen
8,A Fall from Grace,"Tyler Perry (director); Crystal R. Fox, Phylic...",Thriller,Tyler Perry,Tyler Perry (director); Crystal R. Fox,Phylicia Rashad,Bresha Webb
9,The Gentlemen,Guy Ritchie (director/screenplay); Matthew McC...,Action Crime Comedy,Guy Ritchie,Matthew McConaughey,Charlie Hunnam,Henry Golding


In [91]:
df_2020 = df_2020.rename(columns={'Title':'movie_title'})

In [92]:
new_df20 = df_2020.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','Genres','movie_title']]

In [93]:
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,Genres,movie_title
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery,The Grudge
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Action Horror Science Fiction Thriller,Underwater
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,Like a Boss
3,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Drama Thriller Crime,Inherit the Viper
4,Andrew Desmond,Freya Tingley,Simon Abkarian,Rutger Hauer,Horror Thriller Mystery,The Sonata
5,Daniel Farrands,Mena Suvari,Nick Stahl,Taryn Manning,Thriller Drama Crime,The Murder of Nicole Brown Simpson
6,Adil El Arbi and Bilall Fallah,Will Smith,Martin Lawrence,Vanessa Hudgens,Thriller Action Crime,Bad Boys for Life
7,Stephen Gaghan,Robert Downey Jr.,Antonio Banderas,Michael Sheen,Comedy Fantasy Adventure Family,Dolittle
8,Tyler Perry,Tyler Perry (director); Crystal R. Fox,Phylicia Rashad,Bresha Webb,Thriller,A Fall from Grace
9,Guy Ritchie,Matthew McConaughey,Charlie Hunnam,Henry Golding,Action Crime Comedy,The Gentlemen


In [96]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['Genres']

In [97]:
new_df20.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      2
actor_3_name     22
Genres            1
movie_title       0
comb             23
dtype: int64

In [98]:
new_df20 = new_df20.dropna(how='any')

In [99]:
new_df20.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
Genres           0
movie_title      0
comb             0
dtype: int64

In [100]:
new_df20['movie_title'] = new_df20['movie_title'].str.lower()

C:\Users\sam16\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [101]:
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,Genres,movie_title,comb
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery,the grudge,Andrea Riseborough Demián Bichir John Cho Nico...
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Action Horror Science Fiction Thriller,underwater,Kristen Stewart Vincent Cassel Jessica Henwick...
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,like a boss,Tiffany Haddish Rose Byrne Salma Hayek Miguel ...
3,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Drama Thriller Crime,inherit the viper,Josh Hartnett Margarita Levieva Chandler Riggs...
4,Andrew Desmond,Freya Tingley,Simon Abkarian,Rutger Hauer,Horror Thriller Mystery,the sonata,Freya Tingley Simon Abkarian Rutger Hauer Andr...
5,Daniel Farrands,Mena Suvari,Nick Stahl,Taryn Manning,Thriller Drama Crime,the murder of nicole brown simpson,Mena Suvari Nick Stahl Taryn Manning Daniel Fa...
6,Adil El Arbi and Bilall Fallah,Will Smith,Martin Lawrence,Vanessa Hudgens,Thriller Action Crime,bad boys for life,Will Smith Martin Lawrence Vanessa Hudgens Adi...
7,Stephen Gaghan,Robert Downey Jr.,Antonio Banderas,Michael Sheen,Comedy Fantasy Adventure Family,dolittle,Robert Downey Jr. Antonio Banderas Michael She...
8,Tyler Perry,Tyler Perry (director); Crystal R. Fox,Phylicia Rashad,Bresha Webb,Thriller,a fall from grace,Tyler Perry (director); Crystal R. Fox Phylici...
9,Guy Ritchie,Matthew McConaughey,Charlie Hunnam,Henry Golding,Action Crime Comedy,the gentlemen,Matthew McConaughey Charlie Hunnam Henry Goldi...


In [102]:
old_df = pd.read_csv('datasets/final_data.csv')

In [103]:
old_df.shape

(5868, 7)

In [104]:
final_df = old_df.append(new_df20,ignore_index=True)

In [105]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,Genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unkown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unkown Doug Walker Docu...
5,Andrew Stanton,Daryl Sabara,Samantha Morton,Polly Walker,Action Adventure Sci-Fi,john carter,Daryl Sabara Samantha Morton Polly Walker Andr...
6,Nathan Greno,Brad Garrett,Donna Murphy,M.C. Gainey,Adventure Animation Comedy Family Fantasy Musi...,tangled,Brad Garrett Donna Murphy M.C. Gainey Nathan G...
7,Joss Whedon,Chris Hemsworth,Robert Downey Jr.,Scarlett Johansson,Action Adventure Sci-Fi,avengers: age of ultron,Chris Hemsworth Robert Downey Jr. Scarlett Joh...
8,David Yates,Alan Rickman,Daniel Radcliffe,Rupert Grint,Adventure Family Fantasy Mystery,harry potter and the half-blood prince,Alan Rickman Daniel Radcliffe Rupert Grint Dav...
9,Zack Snyder,Henry Cavill,Lauren Cohan,Alan D. Purwin,Action Adventure Sci-Fi,batman v superman: dawn of justice,Henry Cavill Lauren Cohan Alan D. Purwin Zack ...


In [106]:
final_df.to_csv('datasets/main_data.csv',index=False)

In [107]:
final_df.shape

(6067, 7)